Operator Pattern am Beispiel MariaDB
------------------------------------

[MariaDB](https://mariadb.org/) ist ein freies, relationales Open-Source-Datenbankmanagementsystem, das durch eine Abspaltung aus MySQL entstanden ist

Der [MariaDB Operator](https://operatorhub.io/operator/mariadb-operator-app) erweitert Kubernetes um [CustomResourceDefinition](https://kubernetes.io/docs/tasks/extend-kubernetes/custom-resources/custom-resource-definitions/) für MariaDB.

Die neu erstellten [CustomResource Definitionen](https://kubernetes.io/docs/tasks/extend-kubernetes/custom-resources/custom-resource-definitions/) sind:
* MariaDB Server - Verwaltung der Datenbank
* MariaDB Backup - Backup der Datenbank
* MariaDB Monitor - Monitoring der Datenbank

### Installation MariaDB Operator

In [ ]:
%%bash
export NAMESPACE=mariadb
git clone https://github.com/abalki001/mariadb-operator.git
cd mariadb-operator
echo ....... Creating namespace ....... 
kubectl create namespace ${NAMESPACE}
echo ....... Applying CRDs .......
kubectl apply -f deploy/crds/mariadb.persistentsys_mariadbs_crd.yaml -n ${NAMESPACE}
kubectl apply -f deploy/crds/mariadb.persistentsys_backups_crd.yaml -n ${NAMESPACE}
kubectl apply -f deploy/crds/mariadb.persistentsys_monitors_crd.yaml -n ${NAMESPACE}
echo ....... Applying Rules and Service Account .......
kubectl apply -f deploy/service_account.yaml  -n ${NAMESPACE}
kubectl apply -f deploy/role.yaml -n ${NAMESPACE}
sed -i 's/.*namespace:.*/    namespace: ${NAMESPACE}/' deploy/role_binding.yaml
kubectl apply -f deploy/role_binding.yaml  -n ${NAMESPACE}
echo ....... Applying Operator .......
kubectl apply -f deploy/operator.yaml -n ${NAMESPACE}
echo ....... Creating the CRs .......
kubectl apply -f deploy/crds/mariadb.persistentsys_v1alpha1_mariadb_cr.yaml -n ${NAMESPACE}
kubectl apply -f deploy/crds/mariadb.persistentsys_v1alpha1_backup_cr.yaml -n ${NAMESPACE}
kubectl apply -f deploy/crds/mariadb.persistentsys_v1alpha1_monitor_cr.yaml -n ${NAMESPACE}

cat <<%EOF% | kubectl apply -n ${NAMESPACE} -f - 
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: mariadb-pv-claim
spec:
  storageClassName: local-storage
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 1Gi
%EOF%


In [ ]:
! kubectl get all --namespace mariadb

- - - 

Nach erfolgreicher Installation des MariaDB Operators können wir nun neue Datenbank erstellen.

Dazu verwenden wir die neu erstellen [CustomResourceDefinition](https://kubernetes.io/docs/tasks/extend-kubernetes/custom-resources/custom-resource-definitions/)

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: mariadb.persistentsys/v1alpha1
kind: MariaDB
metadata:
  name: mariadb
spec:
  # Keep this parameter value unchanged.
  size: 1
  
  # Root user password
  rootpwd: password

  # New Database name
  database: test-db
  # Database additional user details (base64 encoded)
  username: db-user 
  password: db-user 

  # Image name with version
  image: "mariadb/server:10.3"

  # Database storage Path
  dataStoragePath: "/data/mariadb" 

  # Database storage Size (Ex. 1Gi, 100Mi)
  dataStorageSize: "1Gi"

  # Port number exposed for Database service
  port: 30685
---
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: mariadb-pv-claim
spec:
  storageClassName: local-storage
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 1Gi
%EOF%

In [ ]:
! kubectl get MariaDB --all-namespaces 